In [ ]:
import glob
import os
import pandas as pd
import numpy as np

## Run this for the HK2 samples

In [1]:
!mkdir -p ./2.Re_do_with_lambda_macs 

#### Write the pbs parameters for macs

In [52]:
# configure the parameters
nodes_ppn = [1, 1]
memory = 16
wait_time = [20, 00]
quene = "medium"
inpath = '/mount/weili3/xc3/ENL2_ChIP/data/'
exps = ['293*', 'HK2*']
ctrl = 'input*'
treat = 'F*'
outpath = '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/'
exp = exps[0]

### write the batches of bash jobs

In [2]:
!ls /mount/weili3/xc3/ENL2_ChIP/res_nsp/*.wig.gz

/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_ENL.nsp_control_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_ENL.nsp_treat_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_T1.nsp_control_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_T1.nsp_treat_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_T2.nsp_control_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_T2.nsp_treat_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_T3.nsp_control_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_T3.nsp_treat_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_Y78A.nsp_control_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/293_F_Y78A.nsp_treat_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/HK2_F_ENL.nsp_control_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/HK2_F_ENL.nsp_treat_afterfiting_all.wig.gz
/mount/weili3/xc3/ENL2_ChIP/res_nsp/HK2_F_T1

In [6]:
treat_files = glob.glob(inpath+exp+treat+'/*.bowtie.bed.nsp.shuf')
ctrl_file = glob.glob(inpath+exp+ctrl+'/*.bowtie.bed.nsp.shuf')[0]
print(treat_files)
print(ctrl_file)

['/mount/weili3/xc3/ENL2_ChIP/data/293_F_T1/293_F_T1.bowtie.bed.nsp.shuf', '/mount/weili3/xc3/ENL2_ChIP/data/293_F_T2/293_F_T2.bowtie.bed.nsp.shuf', '/mount/weili3/xc3/ENL2_ChIP/data/293_F_T3/293_F_T3.bowtie.bed.nsp.shuf', '/mount/weili3/xc3/ENL2_ChIP/data/293_F_Y78A/293_F_Y78A.bowtie.bed.nsp.shuf', '/mount/weili3/xc3/ENL2_ChIP/data/293_F_ENL/293_F_ENL.bowtie.bed.nsp.shuf']
/mount/weili3/xc3/ENL2_ChIP/data/293_input_S32/293_input_S32.bowtie.bed.nsp.shuf


In [7]:
# open the initialization header for pbs and read the contents
with open('/home/xc3/experiment/initial.pbs','r') as f:
	pbs_header = f.readlines()

for exp in exps:
	
	treat_files = glob.glob(inpath+exp+treat+'/*.bowtie.bed.nsp.shuf')
	ctrl_file = glob.glob(inpath+exp+ctrl+'/*.bowtie.bed.nsp.shuf')[0]
	names = [ifile.split('/')[-2] for ifile in treat_files]

	for i, treat_file in enumerate(treat_files):
		experiment_name = "macs_{}.ato".format(names[i])
		with open(experiment_name,'w') as f:

			pbs_initial = pbs_header[:]
			# configuration for the experiments
			# pbs_initial[1]::job name
			pbs_initial[1] = pbs_initial[1].format(str(names[i]))
			# pbs_initial[2]::nodes and ppn
			pbs_initial[2] = pbs_initial[2].format(*nodes_ppn)
			# pbs_initial[4]::memeroy
			pbs_initial[4] = pbs_initial[4].format(memory)
			# pbs_initial[5]::waiting time
			pbs_initial[5] = pbs_initial[5].format(*wait_time)
			# pbs_initial[11]::err output
			pbs_initial[11] = pbs_initial[11].format(names[i])
			# pbs_initial[12]::log
			pbs_initial[12] = pbs_initial[12].format(names[i])
			# pbs_initial[14]::quene
			pbs_initial[14] = pbs_initial[14].format(quene)

			
			# write all the configurations into the pbs file
			for line in pbs_initial:
				f.write(line)

			# write the shuf code
			line ="macs14 -t {} -c {} -n {}.nsp --nomodel -g hs --wig -S -p 1e-8".format(treat_files[i], ctrl_file, outpath + names[i])
			
			f.write(line)

### Annotated the above peaks 

In [30]:
def annoate_bed_to_gene(infile, outfile, cutoff=5000):
    ref, tss_set = {}, set()
    reffile = '/mount/weili3/xc3/genomes/hg19.refGene.txt'
    for line in open(reffile):
        col = line.split('\t')
        name, cr, strand, TSS, TES, symbol = col[1], col[2], col[3], int(col[4]), int(col[5]), col[12]
        if strand == '-': TSS, TES = TES, TSS
        if cr not in ref: ref[cr] = []
        if (cr,TSS,strand) not in tss_set:
            ref[cr].append((name,symbol,strand,TSS,TES))
            tss_set.add((cr,TSS,strand))
    for cr in ref: ref[cr].append(('none','none','none',0,0))
    
    # annotate the file
    text = open(infile).readlines()
    fout = open(outfile, 'w')
    print('processing on {}\n will output {}\n'.format(infile,outfile))
    for line in text:
        col = line.split('\t')
        try: cr, start, end = col[0], int(col[1]), int(col[2])
        except: 
            fout.write(line[:-1]+'\twithin_genebody\tnearest_TSS\tdistance\n')
            continue
        if cr not in ref: continue
        peak = (start + end) / 2
        genes, genebody, genes0, genebody0 = [], [], [], []
        for name, symbol, strand, TSS, TES in ref[cr]:
            if strand == '+':
                dist = end - TSS
                if abs(start-TSS)<abs(dist):dist=start-TSS

            elif strand == '-':
                dist = TSS - end
                if abs(TSS - start)<abs(dist):dist=TSS - start

            elif strand != 'none': raise ValueError
            if abs(dist) <= cutoff: genes.append((abs(dist),symbol,dist)) 
            if (start - TSS) * (start - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (end - TSS) * (end - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (start - TSS) * (end - TES) <=0: genebody.append((abs(dist),symbol,dist))
        genes, genebody = sorted(genes), sorted(genebody)
        for g in sorted(genes):
            if g[1] not in [x[1] for x in genes0]: genes0.append(g)
        for g in sorted(genebody):
            if g[1] not in [x[1] for x in genebody0]: genebody0.append(g)
        if any(genes0):
            symbols = ','.join([x[1] for x in genes0])
            dists = ','.join(['%d' % x[2] for x in sorted(genes0)])
        else: symbols, dists = 'none', 'none'
        if any(genebody): body = ','.join([x[1] for x in genebody0])
        else: body = 'none'
        fout.write(line[:-1] + '\t%s\t%s\t%s\n' % (body,symbols,dists))
    fout.close()    

In [5]:
!ls $outpath/*.anno*.txt
# !tail /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed

/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_ENL.nsp_peaks.anno.1k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_ENL.nsp_peaks.anno.3k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_ENL.nsp_peaks.anno.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T1.nsp_peaks.anno.1k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T1.nsp_peaks.anno.3k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T1.nsp_peaks.anno.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T2.nsp_peaks.anno.1k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T2.nsp_peaks.anno.3k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T2.nsp_peaks.anno.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T3.nsp_peaks.anno.1k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T3.nsp_peaks.anno.3k.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_T3.nsp_peaks.anno.txt
/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda//293_F_Y78A.nsp_peaks.anno.1k.txt
/mount/weili3/xc3/ENL2_ChIP/res_

In [53]:
glob.glob(f'/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2*nsp_peaks.bed')
outpath

'/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/'

In [42]:
files_to_annoate = glob.glob(f'/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2*nsp_peaks.bed')
files_to_annoate
infile = files_to_annoate[0]
for infile in files_to_annoate:
    outfile = outpath + infile.split('/')[-1].split('.bed')[0] + '.anno.txt'
    annoate_bed_to_gene(infile, outfile)

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed
 will output ./2.Re_do_with_lambda_macs/HK2_F_T2.nsp_peaks.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.bed
 will output ./2.Re_do_with_lambda_macs/HK2_F_Y78A.nsp_peaks.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed
 will output ./2.Re_do_with_lambda_macs/HK2_F_T1.nsp_peaks.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed
 will output ./2.Re_do_with_lambda_macs/HK2_F_T3.nsp_peaks.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed
 will output ./2.Re_do_with_lambda_macs/HK2_F_ENL.nsp_peaks.anno.txt



## Extract the signal for the peaks
2018-7-5
bed files locates: /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/
bw files locates: /home/xc3/output/ENL2/ChIP/bw_nsp/

### 1. merge the bedfiles together for Treat and wildtype (ENL)

In [57]:
exps = ['293', 'HK2']
bwpath = '/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/'
exp = exps[1]
treats = ['T1', 'T2', 'T3', 'Y78A']
wt = 'ENL'
inpath = '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/'
outpath = './2.Re_do_with_lambda_macs/'

In [62]:
# merge a list of the bed files
# depeand on the bedtools
def merge_beds(bed_files, out_name, header=True):
    """
    bed_files: a list of bed files to be merged
    """
    lines = []
    for bed_file in bed_files:
        with open(bed_file, 'r') as f:
            lines.extend(f.readlines()[1:] if header else f.readlines())
    # merge the two beds together        
    with open('tmp.bed','w') as f:
        f.writelines(lines)
    
    # sort the files
    command = "sort -k1,1 -k2,2n tmp.bed > tmp.sorted.bed"
    print(f'execute {command}')
    os.system(command)
    
    # merge the files
    command = f"bedtools merge -i tmp.sorted.bed -d 1 > {out_name}"
    print(f'execute {command}')
    os.system(command)

In [63]:
#293_F_T2.nsp_peaks.bed
# for exp in exps:
for treat in treats:
    treat_bed = inpath + f'{exp}_F_{treat}.nsp_peaks.bed'
    wt_bed = inpath + f'{exp}_F_ENL.nsp_peaks.bed'
    bed_files = [treat_bed, wt_bed]
    print(bed_files)
    out_name = outpath + f'{exp}_{treat}_WT.merge.bed'
    merge_beds(bed_files, out_name=out_name, header=False)

['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed']
execute sort -k1,1 -k2,2n tmp.bed > tmp.sorted.bed
execute bedtools merge -i tmp.sorted.bed -d 1 > ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.bed
['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed']
execute sort -k1,1 -k2,2n tmp.bed > tmp.sorted.bed
execute bedtools merge -i tmp.sorted.bed -d 1 > ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.bed
['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed']
execute sort -k1,1 -k2,2n tmp.bed > tmp.sorted.bed
execute bedtools merge -i tmp.sorted.bed -d 1 > ./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.bed
['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed']


In [15]:
# function for formating the chr start end into bed format
# if the last column has sign ('+' or '-'), then prepare to swap tss and tes for '-'
def convert_region_to_bed(bedfile, outname, header=None):
    df = pd.read_csv(bedfile,sep='\t',header=None)
    df.columns = np.arange(0,df.shape[1])
    # detect the strand and swap the start end when negative strand
    # the last column should be strand
    if df.iloc[0,-1] == '-' or df.iloc[0,-1] == '+':
        idx = df.iloc[:,-1] == '-'
        # swap the tss tes according to the strand
        df.loc[idx,[1,2]] = df.loc[idx,[2,1]].values
        # modify the strand
        df.iloc[:,-1] = '+'
        
    df.loc[:,3] = np.arange(df.shape[0])# for gene name column
    df.loc[:,4] = 0 # for bed format (value)
    df.loc[:,5] = '+'
    df = df.reindex(range(6), axis='columns')
    print('writing the output --> {}'.format(outname))
    print(f'{outname} --->: reformated bed_file as the input of the bigWigAverageoverBed')
    df.to_csv(outname,sep='\t',index=False,header=None)

In [28]:
def comp_avg_sig_from_bw_by_bed(bedfile, bw_files, which_column=2):
    """
    bedfile: the input merged bedfile which needs to extract the signal from
    bw_files: list of bw files whose signal is extracted from
    which_column: 3 is the acutal counts column, 4 is the acutal avg signal column;
    but, we need to use the first column as index, then 2 is the counts column;
    outname: reformat the bedfile to the normal 6 columns bed file
    will save the output file in the same folder as the bedfile
    """
    outname = bedfile.split('.bed')[0] + '.str.bed'
    # convert the 3 columns bed to the normal bed file
    convert_region_to_bed(bedfile, outname)
    for i,bw_file in enumerate(bw_files):
        command = line = f'~/xc3/software/bigWigAverageOverBed {bw_file} {outname} avg_sig_{i}.txt'
        print('execute --> {}'.format(command))
        os.system(command)
    dfs = []
    for i in range(len(bw_files)):
        dfs.append(pd.read_csv(f'avg_sig_{i}.txt',sep='\t',header=None, index_col=0))
    df = pd.DataFrame([dfs[0].iloc[:,0]] + # iloc[:,0]: the width
                      [dfi.iloc[:,which_column] for dfi in dfs]).T # conbine the columns together
    df.columns = ['width'] + [bw_file.split('/')[-1].split('.')[0] for bw_file in bw_files]
    df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = ['chr', 'start', 'end'])
    df_out = df_bed.join(df)
    print(f"output file -->: {bedfile.split('.bed')[0] + '.counts.txt'}")
    df_out.to_csv(bedfile.split('.bed')[0] + '.counts.txt', sep='\t', index=False)


In [67]:
treats = ['T1', 'T2', 'T3', 'Y78A']
treats = ['T1', 'Y78A']

### 2. extract the signal
merged bed files : out_name = outpath + f'{exp}_{treat}_WT.merge.bed'
'./res_avg_signal/293_T1_WT.merge.bed'

In [68]:
for treat in treats:
    bedfile = outpath + f'{exp}_{treat}_WT.merge.bed'
    df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = ['chr', 'start', 'end'])
    bw_files = [glob.glob(bwpath+exp+'*'+tmp+'*.bw')[0] for tmp in [treat,'ENL', 'ctrl']]
    print(f'bw_files--->:\n{"|".join(bw_files)}')
    outname = bedfile.split('.bed')[0] + '.test.bed'
    print(f'outname--->: {outname}')
    convert_region_to_bed(bedfile, outname) #format the bedfile as outname, especially for the bigWigAve
    comp_avg_sig_from_bw_by_bed(bedfile, bw_files, which_column=2)

bw_files--->:
/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw|/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw|/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_ctrl.nsp.bw
outname--->: ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.test.bed
writing the output --> ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.test.bed
./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.test.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
writing the output --> ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed
./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed avg_sig_0.txt
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed avg_sig_1.tx

In [31]:
### tester code
exp = 'HK2'
bedfile = './2.Re_do_with_lambda_macs/HK2_T1_WT.merge.bed'
df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = ['chr', 'start', 'end'])
bw_files = [glob.glob(bwpath+exp+'*'+tmp+'*.bw')[0] for tmp in [treat,'ENL', 'ctrl']]
print(f'bw_files--->:\n{"|".join(bw_files)}')
outname = bedfile.split('.bed')[0] + '.test.bed'
print(f'outname--->: {outname}')
convert_region_to_bed(bedfile, outname) #format the bedfile as outname, especially for the bigWigAve
comp_avg_sig_from_bw_by_bed(bedfile, bw_files)
df = pd.read_csv(bedfile.split('.bed')[0] + '.counts.txt', sep='\t')
df['width_val'] = df['end'] - df['start']
print(f'results --> \n: {df.head()}')
df['validataion'] = df['width_val'] == df['width'].values.astype(int)
print(sum(df['validataion']))
df[df['validataion']==False]

bw_files--->:
/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_Y78A_treat.nsp.bw|/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw|/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_ctrl.nsp.bw
outname--->: ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.test.bed
writing the output --> ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.test.bed
./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.test.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
writing the output --> ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed
./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_Y78A_treat.nsp.bw ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed avg_sig_0.txt
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.str.bed avg_sig_

,chr,start,end,width,HK2_F_Y78A_treat,HK2_F_ENL_treat,HK2_ctrl,width_val,validataion


### 3. Annotate the signal 

In [69]:
files_to_annoate = glob.glob(f'{outpath}*merge.counts.txt')
files_to_annoate
infile = files_to_annoate[0]
for infile in files_to_annoate:
    outfile = infile.split('.txt')[0] + '.anno.txt'
    annoate_bed_to_gene(infile, outfile)

processing on ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.counts.txt
 will output ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.counts.anno.txt

processing on ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.counts.txt
 will output ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.counts.anno.txt

processing on ./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.counts.txt
 will output ./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.counts.anno.txt

processing on ./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.counts.txt
 will output ./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.counts.anno.txt



### 4. Calculate the signal for each peak
convert the count files to sig files and store them seperately

In [70]:
!head ./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.counts.anno.txt -n2
files_to_annoate = glob.glob(f'{outpath}*merge.counts.txt')
files_to_annoate
for infile in files_to_annoate:
    outfile = infile.split('.counts.txt')[0] + '.sig.txt'
    df = pd.read_csv(infile, header=0, sep='\t')
    df.iloc[:,-3:] = df.iloc[:,-3:].div(df.iloc[:,3],axis=0)
    df.to_csv(outfile, sep='\t', index=False)

chr	start	end	width	HK2_F_Y78A_treat	HK2_F_ENL_treat	HK2_ctrl	within_genebody	nearest_TSS	distance
chr1	804921	805749	828	7271	9846	2373	FAM41C	none	none


In [74]:
## annotate the signal file with the gene annotation
files_to_annoate = glob.glob(f'{outpath}*merge.sig.txt')
files_to_annoate
infile = files_to_annoate[0]
for infile in files_to_annoate:
    outfile = infile.split('.txt')[0] + '.anno.txt'
    annoate_bed_to_gene(infile, outfile)

processing on ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.sig.txt
 will output ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.sig.anno.txt

processing on ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.sig.txt
 will output ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.sig.anno.txt

processing on ./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.sig.txt
 will output ./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.sig.anno.txt

processing on ./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.sig.txt
 will output ./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.sig.anno.txt



### 5. Generate the counts for edgeR
write the count for each paired treat versus ctrl and do the differential analysis

In [76]:
# 1. generate the files with counts
files_to_annoate = glob.glob(f'{outpath}*merge.counts.txt')
files_to_annoate
for infile in files_to_annoate:
    outfile_treat = infile.split('.counts.txt')[0] + '.t.txt'
    outfile_ctrl = infile.split('.counts.txt')[0] + '.c.txt'
    df = pd.read_csv(infile, header=0, sep='\t')
    df.iloc[:,4].to_csv(outfile_treat, sep='\t',index=True,header=False)
    df.iloc[:,5].to_csv(outfile_ctrl, sep='\t',index=True,header=False)
    

In [77]:
# 2. write the code for differential analysis
# edgeR template should run in the expression folder
# run the sh
files_to_annoate = glob.glob(f'{outpath}*merge.counts.txt')
# print(files_to_annoate)
with open('de_pval_counts.sh', 'w') as f:
    for infile in files_to_annoate:
#         print(infile)
        outfile_treat = infile.split('/')[-1].split('.counts.txt')[0] + '.t.txt'
        outfile_ctrl = infile.split('/')[-1].split('.counts.txt')[0] + '.c.txt'
        outfile = infile.split('/')[-1].split('.counts.txt')[0] + '.pval.txt'
        line = f'python /home/jlyu/xc3/software/edgeR.py -s /home/jlyu/xc3/software/edgeR.template -q 1 -t both {outfile_treat}:1:2 {outfile_ctrl}:1:2 {outfile}'
        f.write(line+'\n')

In [78]:
# 3. combine the counts and pval together with counts files
files_to_annoate = glob.glob(f'{outpath}*merge.counts.anno.txt')
files_to_annoate
for infile in files_to_annoate:
    outfile = infile.split('.txt')[0] + '.pval.txt'
    pval_file = infile.split('.counts')[0] + '.pval.txt'
    df_pval = pd.read_csv(pval_file, sep='\t', header=0, index_col='id')
    # according to the index to sort the df_pval
    df_pval.sort_index(axis=0, inplace=True)
    df = pd.read_csv(infile, sep='\t', header=0)
    df_comb = pd.concat([df, df_pval], axis=1)
    df_comb.iloc[:, :-3].to_csv(outfile, sep='\t', index=False)
    print(outfile)
    

./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.counts.anno.pval.txt
./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.counts.anno.pval.txt
./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.counts.anno.pval.txt
./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.counts.anno.pval.txt


In [79]:
# 3. combine the sig and pval together with counts files
files_to_annoate = glob.glob(f'{outpath}*merge.sig.anno.txt')
files_to_annoate
for infile in files_to_annoate:
    outfile = infile.split('.txt')[0] + '.pval.txt'
    pval_file = infile.split('.sig')[0] + '.pval.txt'
    df_pval = pd.read_csv(pval_file, sep='\t', header=0, index_col='id')
    # according to the index to sort the df_pval
    df_pval.sort_index(axis=0, inplace=True)
    df = pd.read_csv(infile, sep='\t', header=0)
    df_comb = pd.concat([df, df_pval], axis=1)
    df_comb.iloc[:, :-3].to_csv(outfile, sep='\t', index=False)
    print(outfile)

./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.sig.anno.pval.txt
./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.sig.anno.pval.txt
./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.sig.anno.pval.txt
./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.sig.anno.pval.txt


In [80]:
# 4. extracted the gene lists with fold change for 1.2 and 1.5
files_to_annoate = glob.glob(f'{outpath}*merge.counts.anno.txt')
files_to_annoate
for infile in files_to_annoate:
    outfile1 = infile.split('.txt')[0] + '.genes1_2.txt'
    outfile2 = infile.split('.txt')[0] + '.genes1_5.txt'
    df = pd.read_csv(infile, sep='\t', header=0)
    df['fc'] = df.iloc[:,4].div(df.iloc[:,5])
    # combine the two dataframes into the list
    dfs_list = [df[df['fc'] >= 1.2], df[df['fc'] >= 1.5]]
    # combine the outfiles names into the list 
    outfiles = [outfile1, outfile2]
    # extract the genes
    for tmp, outfile in zip(dfs_list, outfiles):
        body_tss = []
        body = []
        tss = []
        for cols in tmp.iloc[:,7]:
            eles = cols.split(',')
            for ele in eles:
                if ele and ele != 'none':
                    body.append(ele)

        for cols in tmp.iloc[:,8]:
            eles = cols.split(',')
            for ele in eles:
                if ele and ele != 'none':
                    tss.append(ele)

        body.extend(tss)
        body = list(set(body))
        with open(outfile, 'w') as f:
            f.writelines('\n'.join(body))
            
        print(outfile)



./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.counts.anno.genes1_2.txt
./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.counts.anno.genes1_5.txt
./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.counts.anno.genes1_2.txt
./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.counts.anno.genes1_5.txt
./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.counts.anno.genes1_2.txt
./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.counts.anno.genes1_5.txt
./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.counts.anno.genes1_2.txt
./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.counts.anno.genes1_5.txt


In [49]:
files_to_annoate = glob.glob(f'{outpath}*.counts.anno.txt')
files_to_annoate
infile = files_to_annoate[-2]

In [50]:
df = pd.read_csv(infile, sep='\t', header=0)
df['fc'] = df.iloc[:,4].div(df.iloc[:,5])
dfs_list = [df[df['fc'] >= 1.2], df[df['fc'] >= 1.5]]
dfs_list[0].shape

(9607, 11)